In [11]:
pip install openai==0.28.1

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install llmlingua

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [14]:
import openai
import re
import time
import os
import numpy as np

from tqdm import tqdm
from datasets import load_dataset

In [ ]:
pip install python-dotenv

In [15]:
from dotenv import load_dotenv

# load .env
load_dotenv()

openai.api_key = os.environ.get('OPENAI_API_KEY')

In [16]:
gsm8k = load_dataset('gsm8k', 'main')
validation_index = np.load('./lib_prompt/validation_index.npy')
validation_data = gsm8k['train'].select(validation_index)
gsm8k_test = gsm8k['test']

In [17]:
gsm8k['train'][0]['question']

'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'

In [18]:
gsm8k_test = gsm8k['test']

In [19]:
prompt_complex = open('./lib_prompt/prompt_hardest.txt').read()

In [20]:
print(prompt_complex)

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

In [21]:
pip install tenacity

Note: you may need to restart the kernel to use updated packages.


In [22]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_chain,
    wait_fixed
)

@retry(wait=wait_chain(*[wait_fixed(3) for i in range(3)] +
                        [wait_fixed(5) for i in range(2)] +
                        [wait_fixed(10)]))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

In [23]:
def test_answer(pred_str, ans_str):
    pattern = '\d*\.?\d+'
    pred = re.findall(pattern, pred_str)
    if(len(pred) >= 1):
        # print(pred_str)
        pred = pred[-1]
        gold = re.findall(pattern, ans_str)
        # print(ans_str)
        gold = gold[-1]
        return pred == gold
    else: return False

def parse_pred_ans(filename):
    with open(filename) as fd: lines = fd.readlines()
    am, a = None, None
    num_q, acc = 0, 0
    current_mode = 'none'
    questions = []
    ans_pred = []
    ans_gold = []
    for l in lines:
        if(l.startswith('Q: ')):
            if(am is not None and a is not None):
                questions.append(q)
                ans_pred.append(am)
                ans_gold.append(a)
                if(test_answer(am, a)):
                    acc += 1
            current_mode = 'q'
            q = l
            num_q += 1
        elif(l.startswith('A_model:')):
            current_mode = 'am'
            am = l
        elif(l.startswith('A:')):
            current_mode = 'a'
            a = l
        else:
            if(current_mode == 'q'): q += l
            elif(current_mode == 'am'): am += l
            elif(current_mode == 'a'): a += l
            else:
                raise ValueError(current_mode)

    questions.append(q)
    ans_pred.append(am)
    ans_gold.append(a)
    if(test_answer(am, a)):
        acc += 1
    print('num_q %d correct %d ratio %.4f' % (num_q, acc, float(acc / num_q)))
    return questions, ans_pred, ans_gold

def test_finished(ans_model):
    if('answer is' in ans_model): return True
    else: return False

def extract_ans(ans_model):
    ans_model = ans_model.split('\n')
    ans = []
    residual = []
    for li, al in enumerate(ans_model):
        ans.append(al)
        if('answer is' in al):
            break
    residual = list(ans_model[li + 1:])
    ans = '\n'.join(ans)
    residual = '\n'.join(residual)
    return ans, residual

In [24]:
prompt_q = prompt_complex + '\nQuestion: ' + gsm8k_test[1]['question'] + '\n'

In [25]:
print(prompt_q)

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

In [26]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Follow the given examples and answer the question."},
        {"role": "user", "content": prompt_q},
    ]
)

In [27]:
response['choices'][0]['message']['content']

"To find the total number of bolts needed, we first consider that the robe requires 2 bolts of blue fiber and half that much white fiber. \n\nLet's first find out how many bolts of white fiber are needed:\n\n1. Blue fiber = 2 bolts\n2. White fiber = 1/2 * 2 bolts = 1 bolt\n\nTherefore, the total number of bolts needed for the robe is 2 (blue) + 1 (white) = 3 bolts in total. \n\nThe robe will need 3 bolts in total."

In [28]:
print(response['choices'][0]['message']['content'])

To find the total number of bolts needed, we first consider that the robe requires 2 bolts of blue fiber and half that much white fiber. 

Let's first find out how many bolts of white fiber are needed:

1. Blue fiber = 2 bolts
2. White fiber = 1/2 * 2 bolts = 1 bolt

Therefore, the total number of bolts needed for the robe is 2 (blue) + 1 (white) = 3 bolts in total. 

The robe will need 3 bolts in total.


In [42]:
# prompt_q를 압축해서 시도

from llmlingua import PromptCompressor
llm_lingua = PromptCompressor(device_map="mps")
compressed_prompt = llm_lingua.compress_prompt(prompt_q, target_token=500)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [41]:
print(compressed_prompt['compressed_prompt'])

: Angelo andanie to plan many hours study their test24 to They should study study total week they1 hour,3 breaks and
s think step
anie eachters hours hours
ate1 hours each hours
 andanie to start with planning,
 to. they203ute
unch for  minutes hour3
So andanie study study5 days to for
The4

: Marks team52.  score teams
s,5
 team also63,8
 and throws point
 together0
s scoreds scored4 half5
The is The
: B has two as than she2 more what of have she
s
 have5ella manybles as0
 Bella25 will more than deck, have
+0The A4anas first2 many there


,8
ets3anges theThe combined3 basket4
The1
:13ruits between cost

, then I632
1If
 of is and that price
 can)2

: a large,0,, many the most

,0y total
,:0 Heanged into
0 boxesters0He5 boxes0ese three to3, 1000 - 480 = 520 do not like to play basketball.
The percentage of the school that do not like to play basketball is 520/1000 * 100 = 52
The answer is 52

Question: John hires a driving service to get him to work each day.  His work is 30 miles away

In [38]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Follow the given examples and answer the question."},
        {"role": "user", "content": compressed_prompt['compressed_prompt']},
    ]
)

In [39]:
response['choices'][0]['message']['content']

'John drives a total of 30 miles to work and back each day, so he drives 30 * 2 = 60 miles per day.\nHe goes to work 5 days a week, so he drives 60 * 5 = 300 miles per week.\nHe works for 50 weeks a year, so he drives 300 * 50 = 15,000 miles per year.\nThe cost per mile is $2, so he pays 15,000 * 2 = $30,000 per year for driving.\nHe also gives his driver a $150 bonus per month, which is $150 * 12 = $1800 per year.\nTherefore, John pays a total of $30,000 + $1800 = $31,800 per year for driving.'

In [40]:
print(response['choices'][0]['message']['content'])

John drives a total of 30 miles to work and back each day, so he drives 30 * 2 = 60 miles per day.
He goes to work 5 days a week, so he drives 60 * 5 = 300 miles per week.
He works for 50 weeks a year, so he drives 300 * 50 = 15,000 miles per year.
The cost per mile is $2, so he pays 15,000 * 2 = $30,000 per year for driving.
He also gives his driver a $150 bonus per month, which is $150 * 12 = $1800 per year.
Therefore, John pays a total of $30,000 + $1800 = $31,800 per year for driving.
